# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - System instructions
* System instructions are like a preamble that you add before the LLM gets exposed to any further instructions from the user. It lets users steer the behavior of the model based on their specific needs and use cases.
* For example, you can include things like the role or persona, contextual information, and formatting instructions.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instructions



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 11.3 MB/s eta 0:00:00


In [ ]:
%pip install -q -U pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [ ]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [ ]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)


system_instructions = [
 "Use Korean when responding",
 "Use table format to categorize the facts"
]

# Access to the generative model.
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Function to get the response

In [ ]:
def generate(query:str):
    """
    Generate a response from the model.

    query : str
      query to be sent to the model

    Returns: str
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

## Run example

In [ ]:

from time import perf_counter

t1_start = perf_counter()

prompt = """
You are a Generative AI specialist to help people understand the concept of AI.
Explain what's the pros and cons of using AI in the following instructions.
1. Define what's the Generative AI.
2. What are differences between typical machine learning and Generative AI.
3. What are the pros and cons of using Generative AI.
4. Explain use cases of Generative AI in the financial industry.

"""

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))



Time : 6.920164886000009 seconds




## 생성형 AI에 대한 이해: 장점과 단점

### 1. 생성형 AI란 무엇인가요?

| 용어 | 설명 |
|---|---|
| 생성형 AI | 새로운 콘텐츠를 생성하는 데 중점을 둔 인공지능의 한 유형. 이는 텍스트, 이미지, 오디오, 비디오 및 코드와 같은 다양한 형식의 데이터를 생성할 수 있습니다. |

### 2. 일반적인 머신러닝과 생성형 AI의 차이점은 무엇인가요?

| 유형 | 설명 |
|---|---|
| 일반적인 머신러닝 | 기존 데이터를 사용하여 미래의 결과를 예측하거나 분류하는 데 중점을 둡니다. 예를 들어, 스팸 필터는 이전 데이터를 사용하여 스팸 메일을 식별합니다. |
| 생성형 AI | 새로운 데이터를 생성하는 데 중점을 둡니다. 예를 들어, 이미지 생성 AI는 새로운 이미지를 생성할 수 있습니다. |

### 3. 생성형 AI 사용의 장단점은 무엇인가요?

| 장점 | 단점 |
|---|---|
| **새로운 콘텐츠 생성:** 텍스트, 이미지, 음악 등 다양한 콘텐츠를 생성할 수 있습니다. | **데이터 편향:** 생성된 콘텐츠는 훈련 데이터에 대한 편향을 반영할 수 있습니다. |
| **창의성 향상:** 인간의 창의성을 확장하고 새로운 아이디어를 촉진할 수 있습니다. | **실제성 부족:** 생성된 콘텐츠는 실제 세계와 다를 수 있습니다. |
| **효율성 향상:** 반복적인 작업을 자동화하여 시간과 노력을 절약할 수 있습니다. | **저작권 문제:** 생성된 콘텐츠의 저작권 문제가 발생할 수 있습니다. |
| **개인화된 경험 제공:** 사용자의 선호도에 맞춘 맞춤형 콘텐츠를 제공할 수 있습니다. | **정보 오용 가능성:** 생성된 콘텐츠가 잘못된 정보를 전파하는 데 사용될 수 있습니다. |

### 4. 금융 산업에서 생성형 AI의 활용 사례는 무엇인가요?

| 활용 사례 | 설명 |
|---|---|
| **사기 탐지:** 이상 거래 패턴을 식별하고 사기 행위를 방지하는 데 사용됩니다. |
| **리스크 관리:** 금융 시장의 변동성을 예측하고 위험을 완화하는 데 도움이 됩니다. |
| **고객 서비스 자동화:** 챗봇을 사용하여 고객 질문에 답변하고 지원을 제공합니다. |
| **금융 상품 개발:** 새로운 금융 상품 및 서비스를 개발하는 데 활용됩니다. |
| **자동화된 보고서 작성:** 금융 보고서 및 분석을 자동화하여 효율성을 높입니다. |

**요약:** 생성형 AI는 다양한 분야에서 유용한 기술이지만, 데이터 편향, 실제성 부족, 저작권 문제, 정보 오용 가능성 등과 같은 단점도 있습니다. 이러한 단점을 인지하고 신중하게 사용하는 것이 중요합니다.
